# Settando todo o ambiente caso necessario
Importa bibliotecas, instala a base de dados e aloca as imagens de train e test para arquivos .npy localizados em /content/

In [25]:
import kagglehub
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans, DBSCAN

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score

import matplotlib.pyplot as plt

path = kagglehub.dataset_download("zlatan599/mushroom1") # find it on "kaggle/input" # provavelmente o certo seria so colocar nos arquivos depois de instalado e chega

#print(pd.read_csv(path)) # Isso ta funcionando pro nico? que
mushroomdf_val = pd.read_csv(f"{path}/val.csv") # nao irei usar provavelmente para a analise mas ja deixo ai
mushroomdf_test = pd.read_csv(f"{path}/test.csv") # val kinda related to test
mushroomdf_train = pd.read_csv(f"{path}/train.csv").drop_duplicates(subset=["image_path"])

# Algumas possiveis verificacoes a se fazer (mas visiveis no kaggle)
#mushroomdf_train.describe().T.plot(kind='bar') # no duplicates on images, duplicates on labels
#print(mushroomdf_train.isnull().sum()) # no nulls
#print(mushroomdf_train.duplicated().sum()) # no duplicateds

#from PIL import Image # Pil is too slow tho, probs should just use opencv, am sick of it
import cv2

mushroomdf_val.image_path = mushroomdf_val.image_path.apply( lambda x: os.path.join(path, *x.split('/')[-3:]) )
mushroomdf_train.image_path = mushroomdf_train.image_path.apply( lambda x: os.path.join(path, *x.split('/')[-3:]) )
mushroomdf_test.image_path = mushroomdf_test.image_path.apply( lambda x: os.path.join(path, *x.split('/')[-3:]) )

paths_val = mushroomdf_val.image_path
paths_train = mushroomdf_train.image_path
paths_test = mushroomdf_test.image_path

paths = list(paths_train) + list(paths_test)

# Mais velocidade usando multiprocessamento
from joblib import Parallel, delayed

X_train = []
y_train = []
X_test = []
y_test = []

# These can be found in /content
if os.path.exists("X_train.npy") and os.path.exists("y_train.npy") and os.path.exists("X_test.npy") and os.path.exists("y_test.npy"):
    X_train = np.load("X_train.npy")
    y_train = np.load("y_train.npy", allow_pickle=True)
    X_test = np.load("X_test.npy")
    y_test = np.load("y_test.npy", allow_pickle=True)
    print("Loaded from files")
else:
  print("Loading images...")

  def loadImg(path):
    try:
      img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
      img = cv2.resize(img, (64, 64))
      return img.flatten() / 255.0
    except Exception as e:
      print(f"{e} - Error loading {path}")
      return None

  results = Parallel(n_jobs=-1, prefer="threads")(
      delayed(loadImg)(p)
      for p in paths
  )

  #Removendo cargas falhas
  results = [img for img in results if img is not None]

  X_train = np.array(results[:len(paths_train)])
  y_train = mushroomdf_train.label.to_numpy()

  X_test = np.array(results[len(paths_train):])
  y_test = mushroomdf_test.label.to_numpy()

  # save
  np.save("X_train.npy", X_train)
  np.save("y_train.npy", y_train)
  np.save("X_test.npy", X_test)
  np.save("y_test.npy", y_test)

# ~4 minutos se kaggle nao presente, ~8-10mins pra tudo ser alocado
# ~18segundos-1minuto  se tudo presente

# Algumas verificacoes para ver se ta tudo certo, pode ser comentado
print(f"Total train paths: {len(paths_train)}, total test paths: {len(paths_test)}")
print(f"Images on train loaded: {len(X_train)}, labels loaded: {len(y_train)}, images on test: {len(X_test)}, labels on test: {len(y_test)}")

print(f"Shape of X_train: {X_train.shape} and of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape} and of y_test: {y_test.shape}")

Using Colab cache for faster access to the 'mushroom1' dataset.
Loaded from files
Total train paths: 72858, total test paths: 15614
Images on train loaded: 72858, labels loaded: 72858, images on test: 15614, labels on test: 15614
Shape of X_train: (72858, 4096) and of y_train: (72858,)
Shape of X_test: (15614, 4096) and of y_test: (15614,)


In [26]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=20, random_state=42)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

print("MLP Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Accuracy: 0.10759574740617395
                             precision    recall  f1-score   support

          Agaricus augustus       0.00      0.00      0.00        51
      Agaricus xanthodermus       0.03      0.02      0.02        46
     Amanita amerirubescens       0.00      0.00      0.00        50
            Amanita augusta       0.00      0.00      0.00        50
        Amanita brunnescens       0.00      0.00      0.00        46
      Amanita calyptroderma       0.04      0.06      0.04        49
            Amanita citrina       0.06      0.02      0.03        44
         Amanita flavoconia       0.00      0.00      0.00        48
           Amanita muscaria       0.09      0.46      0.15       687
         Amanita pantherina       0.00      0.00      0.00        51
          Amanita persicina       0.50      0.03      0.05        39
         Amanita phalloides       0.02      0.02      0.02        48
          Amanita rubescens       0.00      0.00      0.00        79

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
